In [ ]:
import pandas
import numpy
from matplotlib import pyplot
filename = "dataframe.csv"
data = pandas.read_csv(filename, header=0, index_col=0).T
data.index = pandas.read_csv(filename, header=None, index_col=0).iloc[0]
data.index.name=""
data.T.plot(legend=None, xlim=[data.columns.max(), data.columns.min()])

In [ ]:
# 横軸補完
flag = False
#flag = True
xdiv = 0.5  # 補完間隔
xmin = data.columns.min() // xdiv * xdiv + xdiv
xmax = data.columns.max()
from scipy import interpolate
ix = numpy.arange(xmin, xmax, xdiv)
buff = pandas.DataFrame(index=data.index, columns=ix)
for i in range(len(data)): buff.iloc[i] = interpolate.interp1d(data.columns, data.iloc[i], kind="quadratic")(ix)
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
# 横軸選択
flag = False
#flag = True
xmax, xmin = 1500, 1000  # 横軸範囲
buff = data.iloc[:, (xmin <= data.columns) & (data.columns <= xmax)]
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
# アウトライヤー除去 (Isolation Forest)
flag = False
#flag = True
frac = 0.05  # 除去する割合
from sklearn.ensemble import IsolationForest
model = IsolationForest(contamination=frac).fit(data.values)
predict = model.predict(data.values)
data[predict == -1].T.plot(legend=None, xlim=[data.columns.max(), data.columns.min()])
data[predict == 1].T.plot(legend=None, xlim=[data.columns.max(), data.columns.min()])
if flag: data = data[predict == 1]

In [ ]:
# アウトライヤー除去 (One Class SVM)
flag = False
#flag = True
frac = 0.05  # 除去する割合
from sklearn.svm import OneClassSVM
model = OneClassSVM(nu=frac).fit(data.values)
predict = model.predict(data.values)
data[predict == -1].T.plot(legend=None, xlim=[data.columns.max(), data.columns.min()])
data[predict == 1].T.plot(legend=None, xlim=[data.columns.max(), data.columns.min()])
if flag: data = data[predict == 1]

In [ ]:
# 平滑化
flag = False
#flag = True
window = 7  # 窓幅
x = data.columns
print("window size =", x[window - 1] - x[0])
y = data.values[-1]
from scipy.signal import savgol_filter
s = savgol_filter(y, window, 2, 0)
pyplot.axes().set_xlim([data.columns.max(), data.columns.min()])
pyplot.scatter(x, y, s=3, c="blue")
pyplot.plot(x, s, c="red")
pyplot.show()
pyplot.axes().set_xlim([data.columns.max(), data.columns.min()])
pyplot.scatter(x, y-s, s=3, c="blue")
pyplot.show()
if flag:
    data = pandas.DataFrame(savgol_filter(data.values, window, 2, 0), index=data.index, columns=data.columns)
    data.T.plot(legend=None, xlim=[data.columns.max(), data.columns.min()])

In [ ]:
# 二次微分
flag = False
#flag = True
window = 7  # 窓幅
print("window size =", x[window - 1] - x[0])
from scipy import signal
buff = data
buff = signal.savgol_filter(buff, window, 2, 0)
buff = signal.savgol_filter(buff, window, 2, 1)
buff = signal.savgol_filter(buff, window, 2, 0)
buff = signal.savgol_filter(buff, window, 2, 1)
buff = pandas.DataFrame(buff, index=data.index, columns=data.columns)
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
# センタリング
flag = False
#flag = True
buff = data - data.mean()
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
# スケーリング
flag = False
#flag = True
buff = data / data.std()
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
# オートスケーリング
flag = False
#flag = True
buff = (data - data.mean()) / data.std()
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
# ノーマライズ
flag = False
#flag = True
#xmax, xmin = 1150, 1000
buff = data.iloc[:, (xmin <= data.columns) & (data.columns <= xmax)]
buff = ((data.T - buff.T.min()) / (buff.T.max() - buff.T.min())).T
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
# 差スペクトル
flag = False
#flag = True
buff = data.iloc[0]  # 基準スペクトル
buff = data - buff
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff

In [ ]:
# 差分スペクトル
flag = False
#flag = True
buff = data.copy()
for i in range(len(data) - 1): buff.iloc[i] = buff.iloc[i + 1] - buff.iloc[i]
buff = buff.iloc[:-1]
buff.T.plot(legend=None, xlim=[buff.columns.max(), buff.columns.min()])
if flag: data = buff